## 5-8. 한국어 문서의 분류

앞에서 영어 문서를 분류하는 다양한 방법을 알아봤다. 한글 문서에 대한 분류는 형태소 분석을 제외하고는 기본적으로 영어 문서의 분류와 동일하다고 할 수 있다. 영어 문서는 사이킷런에서 제공하는 기본 토크나이저를 사용할 수 있었으나, 한글 문서는 처음부터 별도의 토크나이저를 쓸 수밖에 없다. 앞 장에서 배운 KoNLPy를 사용해 한글 문서 분류를 해본다.

### 다음 영화 리뷰에 대한 영화 제목 예측
카은트 벡터 변환의 예제로 사용했던 [다음무비](http://movie.daum.net)의 영화 리뷰를 이용해 분류를 연습해본다. 즉 영화 리뷰의 내용을 분석해서 영화의 제목을 예측하는 분류기를 만든다. 앞 장에서도 데이터 파일의 내용을 간략하게 살펴봤지만, 여기서는 영화 제목과 관련한 내용을 좀 더 들여다보기로 한다.

In [1]:
import pandas as pd
df = pd.read_csv("../data/daum_movie_review.csv")
df.head(5)

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


위 결과에서 title이 영화 제목인 것을 알 수 있는데, 어떤 영화들이 있고, 각 영화에 대해 리뷰의 수는 몇 개나 되는지 알아보기 위해 value_counts() 메서드를 title에 적용해본다.

In [2]:
df.title.value_counts()

신과함께      4947
택시운전사     2322
인피니티 워    2042
범죄도시      1939
곤지암       1547
라라랜드      1150
코코         778
Name: title, dtype: int64

결과를 보면 각 영화별로 리뷰가 몇 개인지를 알 수 있는데, 리뷰의 수가 꽤 차이가 난다. "신과함께"는 4,947개인 반면 "코코"는 778개에 불과하다. 이러한 데이터셋을 불균형 데이터셋(imbalanced dataset)이라고 한다. 이 문제를 해결하기 위한 방법으로, 첫째로 언더샘플링을 통해 갯수가 많은 분류들의 데이터 수를 적은 분류의 수에 맞출 수 있다. 둘째로 오버샘플링을 이용해 데이터 수가 적은 분류의 수를 늘릴 수 있다. 둘 다 나름의 장단점이 있으며, 이 외에도 다른 방법들이 있다. 여기서는 일단 주어진 데이터 수를 그대로 이용해서 분류를 진행해보고자 한다.

20 뉴스그룹과는 달리 다음 영화 리뷰는 학습 데이터셋과 테스트 데이터셋이 분리돼 있지 않다. 따라서 먼저 train_test_split을 이용해 아래와 같이 분리를 해주고, 각 데이터셋의 크기를 확인한다.

In [3]:
from sklearn.model_selection import train_test_split

# 데이터와 라벨을 학습 세트와 평가 세트로 분리, 비율을 지정하지 않으면 75:25로 분할됨.
X_train, X_test, y_train, y_test = train_test_split(df.review, df.title, random_state=0)

print("# Train set size:", len(X_train)) # 실제로 몇 개의 특성이 사용됐는지 확인
print("# Test set size:", len(X_test))

# Train set size: 11043
# Test set size: 3682


이제 데이터가 준비됐으므로 KoNLPy에서 형태소 분석기를 불러온다. 앞 장에서 설명한 바와 같이 KoNLPy에서는 다섯 개의 형태소 분석기를 제공하며, 속도와 성능을 비교한 결과가 [홈페이지](https://konlpy.org/ko/latest/morph/#comparison-between-pos-tagging-classes)에 나와 있다. 자신이 가진 텍스트를 대상으로 직접 형태소 분석을 해보고 가장 잘 맞는 형태소 분석기를 선택하는 것을 추천한다. 다만 이 책에서는 실습 속도를 높이고자 트위터 형태소 분석기를 선택했다.

In [4]:
from konlpy.tag import Okt

okt = Okt()

print(okt.morphs(X_train[1])) # 둘째 리뷰에 대해 형태소 단위로 tokenize
print(okt.nouns(X_train[1])) # 둘째 리뷰에서 명사만 추출

['몰입', '할수밖에', '없다', '.', '어렵게', '생각', '할', '필요없다', '.', '내', '가', '전투', '에', '참여', '한', '듯', '손', '에', '땀', '이남', '.']
['몰입', '생각', '내', '전투', '참여', '듯', '손', '땀', '이남']


위 예제에서는 학습 데이터셋의 두번째 리뷰에 대해 morphs()와 nouns()로 형태소 분석 결과와 그중 명사만 추출한 결과를 확인했다. 일반적으로 문서를 대상으로 분석하는 경우에는 명사만으로도 좋은 결과를 보이는 경우가 많으므로 우선 okt.nouns()를 TfidfVectorizer의 토크나이저로 지정해 로지스틱 회귀분석을 해본다.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Twitter 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

# logistic regression 분류기 선언
# 충분한 학습을 위해 max_iter를 1,000으로 설정 (기본은 100)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train) # 분류기 학습

# train data 예측정확도                                         
print(f"# Train set score: {clf.score(X_train_tfidf, y_train):.3f}")
# test data 예측정확도
print(f"# Test set score: {clf.score(X_test_tfidf, y_test):.3f}")

# Train set score: 0.756
# Test set score: 0.694


클래스의 수가 7개라는 점을 감안할 때, 나쁘지 않은 결과로 보인다. 위 예에서 LogisticRegression 객체를 선언할 때 max_iter 매개변수의 값을 1,000으로 했는데, 이 매개변수의 기본값은 100이고 기본값으로 실행할 경우 충분히 수렴되지 않았다는 warning이 발생해서 권고에 따라 1,000으로 늘렸다.

테스트 데이터에 대해 모형이 어떻게 예측하는지 보기 위해, 테스트 데이터 앞 10개의 리뷰에 대해 실제 영화 제목, 예측한 제목, 리뷰의 순으로 아래와 같이 출력한다.

In [6]:
print("실제영화제목, 예측한 제목, 리뷰")
for content in zip(y_test[:10], clf.predict(X_test_tfidf[:10]), X_test[:10]):
    print(content)

실제영화제목, 예측한 제목, 리뷰
('범죄도시', '신과함께', '오랜만에 잼나는 영화 봤습니다.  다음에 더 재미있는 영화 기대하겠습니다.')
('범죄도시', '범죄도시', '조연들이 눈에 박힌다. 간만에 집중 ㅎ')
('코코', '코코', '대감동을 선사. 인사이드 아웃을 잇는 픽사의 감동스토리. 신과함께의 멕시코판이라고나할까요??')
('신과함께', '신과함께', '돈이 안아까웠던 영화ᆞᆞ  정말 좋았다')
('신과함께', '신과함께', '역시 김용화감독이 영화는 잘 만들어요. 이제 VFX 제작 부문도 헐리우드 수준 이상입니다.')
('택시운전사', '택시운전사', '민주화를 위해 힘써주신 분들께 감사하는 마음으로 살아야겠다.')
('신과함께', '신과함께', '잠만 자다 왔음')
('신과함께', '신과함께', '오랜만에 잼있고 좋은 영화를 봤다')
('범죄도시', '신과함께', '잼남')
('범죄도시', '인피니티 워', '대박~~')


예측이 틀린 리뷰에 대해서 리뷰의 내용을 살펴보면, 사람이 봐도 맞히기 어려울 거 같다는 생각이 든다. '잼남', '대박~~'으로 영화 제목을 맞히기는 아마도 불가능할 것이다. 오히려 '오랜만에 잼있고 좋은 영화를 봤다'로 제목을 맞힌 것이 용한 것 같다. 이 정도면 상당히 잘 맞힌다고 봐도 될 것이다.

### 성능을 개선하기 위한 노력
이제 영어 문서에 대한 분류와 마찬가지로 한글에 대해서도 성능을 개선하기 위한 노력을 해본다. 시작은 형태소 분석이다. 위 예에서는 명사만을 사용했는데 만일 모든 품사를 다 사용한다면 어떨까?

In [7]:
# 명사 대신 모든 형태소를 사용
tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# 충분한 학습을 위해 max_iter를 1,000으로 설정, 기본은 100
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

# train data 예측정확도
print(f"# Train set score: {clf.score(X_train_tfidf, y_train):.3f}")
# test data 예측정확도
print(f"# Test set score: {clf.score(X_test_tfidf, y_test):.3f}")

# Train set score: 0.777
# Test set score: 0.695


0.694에서 0.695로 바뀌었는데, 이 정도로는 향상됐다고 보기 어렵다. 기왕 품사 태깅을 했으니, 전체를 다 사용하는 대신 명사, 동사, 형용사만 선별한다면 어떻게 될까? 이를 위해서 아래와 같이 twit_tokenizer 함수를 정의하고, 정의된 함수를 이용해 특성 추출을 한 후 분류해본다.

In [8]:
def twit_tokenizer(text): # 전체를 다 사용하는 대신 명사, 동사, 형용사를 사용
    target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in okt.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
    return result

# 명사, 동사, 형용사를 이용해 tfidf 생성
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

print(f"# Train set score: {clf.score(X_train_tfidf, y_train):.3f}")
print(f"# Test set score: {clf.score(X_test_tfidf, y_test):.3f}")

# Train set score: 0.784
# Test set score: 0.712


0.695에서 0.712로의 변화는 상당히 차이가 있어 보인다. 즉 품사를 선별하는 것이 도움이 된다는 뜻이다. 모든 품사를 다 사용할 때는 무엇이 문제일지 생각해보면, 같은 단어가 서로 다른 품사로 사용된 경우 이를 구분하지 못한다는 것이다. 동일 단어에 대해 품사를 구분할 수 있도록 많이 쓰는 방법 중 하나가 단어에 품사명을 붙여서 하나의 단어로 만드는 것이다. 컴퓨터 입장에서는 단어를 다른 것들과 구분할 수 있기만 하면 되므로 우리가 보기에 어색한 것은 별 문제가 되지 않는다. 아래 예에서는 이것을 반영한 토크나이저를 정의하고 학습 데이터 중 둘째 리뷰에 대해 적용한 결과를 보인다.

In [9]:
# 모든 형태소를 다 사용하고 품사를 알 수 있게 하면?
def twit_tokenizer2(text):
    result = []
    for word, tag in okt.pos(text, norm=True, stem=True):
        result.append('/'.join([word, tag])) # 단어의 품사를 구분할 수 있게 함
    return result

print(twit_tokenizer2(X_train[1]))

['몰입/Noun', '하다/Verb', '없다/Adjective', './Punctuation', '어렵다/Adjective', '생각/Noun', '하다/Verb', '필요없다/Adjective', './Punctuation', '내/Noun', '가/Josa', '전투/Noun', '에/Josa', '참여/Noun', '한/Determiner', '듯/Noun', '손/Noun', '에/Josa', '땀/Noun', '이남/Noun', './Punctuation']


위 결과를 보면 어떤 식으로 단어가 만들어지는지 이해할 수 있을 것이다. 이제 이 토크나이저를 이용해 분류를 해본다.

In [10]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer2, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

print(f"# Train set score: {clf.score(X_train_tfidf, y_train):.3f}")
print(f"# Test set score: {clf.score(X_test_tfidf, y_test):.3f}")

# Train set score: 0.789
# Test set score: 0.718


결과가 더 나아진 것을 볼 수 있다. 명사, 동사, 형용사만 선택하고 거기에 품사를 붙이면 더 나이질 수 있을까? 이는 직접 해보며 알아보기를 추천한다. 지금까지는 로지스틱 회귀분석의 결과이므로 혹시 릿지 회귀분석을 해보면 더 좋아질 수 있는지 보기 위해 다시 한번 어설픈 그리드 서치를 해본다.

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier

X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

max_score = 0
max_alpha = 0
for alpha in np.arange(0.1, 10, 0.1): # alpha를 0.1부터 10까지 0.1씩 증가
    ridge_clf = RidgeClassifier(alpha=alpha) # 릿지 분류기 선언
    ridge_clf.fit(X_train_ridge, y_train_ridge) # 학습
    score = ridge_clf.score(X_val_ridge, y_val_ridge) # 검정 데이터셋에 대해 정확도를 측정
    if score > max_score: # 정확도가 이전의 정확도 최댓값보다 크면 최댓값을 변경한다.
        max_score = score
        max_alpha = alpha

print(f"# Max alpha {max_alpha:.3f} at max validation score {max_score:.3f}")

# Max alpha 1.600 at max validation score 0.727


이 alpha 값을 이용해 아래와 같이 릿지 회귀분석을 해보고 내친 김에 라쏘 회귀분석도 해본다.

In [12]:
ridge_clf = RidgeClassifier(alpha=1.6)
ridge_clf.fit(X_train_tfidf, y_train)
print(f"# Ridge Train set score: {ridge_clf.score(X_train_tfidf, y_train):.3f}")
print(f"# Ridge Test set score: {ridge_clf.score(X_test_tfidf, y_test):.3f}")

# Ridge Train set score: 0.807
# Ridge Test set score: 0.726


In [13]:
from sklearn.linear_model import LogisticRegression
import numpy as np

lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=0.5)
lasso_clf.fit(X_train_tfidf, y_train)
print(f"# Lasso Train set score: {lasso_clf.score(X_train_tfidf, y_train):.3f}")
print(f"# Lasso Test set score: {lasso_clf.score(X_test_tfidf, y_test):.3f}")

# Lasso Train set score: 0.703
# Lasso Test set score: 0.696


In [14]:
print(f"# Used features count: {np.sum(lasso_clf.coef_ != 0)} out of {X_train_tfidf.shape[1]}")

# Used features count: 957 out of 2000


릿지 회귀분석은 정확도 0.726으로 확실히 더 나은 결과를 보인다. 반면 라쏘 회귀분석은 0.696으로 성능이 떨어진다. 어느 정도 예견했던 결과다. 마지막으로 영어 문서 분류에서 나이브 베이즈가 가장 좋은 성능을 보여줬으니 나이브 베이즈로도 분류해본다.

In [15]:
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB(alpha=0.1)
NB_clf.fit(X_train_tfidf, y_train)

print(f"# Train set score: {NB_clf.score(X_train_tfidf, y_train):.3f}")
print(f"# Test set score: {NB_clf.score(X_test_tfidf, y_test):.3f}")

# Train set score: 0.768
# Test set score: 0.704


생각과는 달리 로지스틱 회귀분석보다도 더 낮은 결과를 보인다. 나이브 베이즈 분석이 항상 좋은 결과를 보이는 것은 아니라는 사실을 알 수 있다. 최적의 성능을 얻으려면 다양한 모형과 세팅을 모두 시도해볼 수밖에 없다. 예를 들어 한국어 문서에서는 max_features를 2,000으로 제한하는데 영어 문서에서와 같이 모든 단어를 다 특성으로 사용하면 성능이 더 나아질까에 대한 답도 직접 해보며 알아보기를 추천한다.